In [54]:
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import Input, layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

# Math
import random as rn
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile

import sounddevice as sd

import pandas as pd
import tensorflow as tf
import glob

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

import IPython.display as ipd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import style
import seaborn as sns

%matplotlib inline
sns.set(style='whitegrid', color_codes=True)

# Pre processing

# Deep learning

In [55]:
seed = 100
np.random.seed(seed)
rn.seed(seed)
tf.set_random_seed(seed)

In [56]:
labels = pd.read_csv('train_labels.csv')
sample_solution = pd.read_csv('sample_submission.csv')
train = np.load('train_data.npy')
test = np.load('test_data.npy')

In [71]:
print(train.shape)
print(test.shape)
print(labels.shape)

(3999, 44100)
(3997, 44100)
(3999, 2)


In [72]:
# sampling frequency
fs = 44100
#sd.play(train[6], fs)
sd.play(test[34], fs)

In [191]:
# train: we see 3999 records of data with sampling frequency of 44100
# test: we see 3997 records of data with sampling frequency of 44100

drop_labels = labels.drop('Id',1)
Y = to_categorical(drop_labels ,2)

print(Y[2])

train_2d = train.reshape(-1,210,210,1)
test_2d = test.reshape(-1,210,210,1)

x_train, x_test, y_train, y_test = train_test_split(
    train_2d, Y, test_size=0.15, random_state=seed)


print("TRAIN SHAPE")
print(x_train.shape, y_train.shape)
print("TEST SHAPE")
print(x_test.shape, y_test.shape)

[1. 0.]
TRAIN SHAPE
(3399, 210, 210, 1) (3399, 2)
TEST SHAPE
(600, 210, 210, 1) (600, 2)


In [192]:
n_timesteps, n_features, n_outputs = x_train.shape[0], x_train.shape[1], y_train.shape[0]
print('# of samples / timestamps:', n_timesteps)
print('frequency    / features  :', n_features)
print('# of outputs / outputs   :', n_outputs)

# of samples / timestamps: 3399
frequency    / features  : 210
# of outputs / outputs   : 3399


In [248]:
# create convolution neural network
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size=(3, 3),
                padding = 'same',
                activation = 'tanh',
                input_shape=(210,210,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters = 16, kernel_size = (3,3), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters = 32, kernel_size = (3,3), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters = 32, kernel_size = (3,3), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(64, activation='tanh')),
model.add(Dense(32, activation='tanh')),
model.add(Dense(2, activation='softmax'))

In [249]:
model.summary()

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_150 (Conv2D)          (None, 210, 210, 16)      160       
_________________________________________________________________
max_pooling2d_112 (MaxPoolin (None, 105, 105, 16)      0         
_________________________________________________________________
conv2d_151 (Conv2D)          (None, 103, 103, 16)      2320      
_________________________________________________________________
max_pooling2d_113 (MaxPoolin (None, 51, 51, 16)        0         
_________________________________________________________________
conv2d_152 (Conv2D)          (None, 49, 49, 32)        4640      
_________________________________________________________________
max_pooling2d_114 (MaxPoolin (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_153 (Conv2D)          (None, 22, 22, 32)      

In [250]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

In [251]:
epochs, batch_size = 35, 147
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
# fit network
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_data=(x_test, y_test))

Train on 3399 samples, validate on 600 samples
Epoch 1/35
3399/3399 [==============================] - 4s 1ms/sample - loss: 0.3017 - acc: 0.8605 - val_loss: 0.1971 - val_acc: 0.9167
Epoch 2/35
3399/3399 [==============================] - 3s 759us/sample - loss: 0.2064 - acc: 0.9109 - val_loss: 0.1824 - val_acc: 0.9267
Epoch 3/35
3399/3399 [==============================] - 3s 755us/sample - loss: 0.1830 - acc: 0.9170 - val_loss: 0.1784 - val_acc: 0.9283
Epoch 4/35
3399/3399 [==============================] - 3s 761us/sample - loss: 0.1608 - acc: 0.9259 - val_loss: 0.1629 - val_acc: 0.9367
Epoch 5/35
3399/3399 [==============================] - 3s 760us/sample - loss: 0.1418 - acc: 0.9420 - val_loss: 0.1560 - val_acc: 0.9317
Epoch 6/35
3399/3399 [==============================] - 3s 760us/sample - loss: 0.1350 - acc: 0.9420 - val_loss: 0.1511 - val_acc: 0.9383
Epoch 7/35
3399/3399 [==============================] - 3s 765us/sample - loss: 0.1213 - acc: 0.9435 - val_loss: 0.1416 - val_a

In [252]:
score = model.evaluate(x_test, y_test, batch_size=147)

600/600 [==============================] - 0s 354us/sample - loss: 0.1236 - acc: 0.9650


In [253]:
final_predictions = model.predict_classes(test_2d)

In [254]:
sample_solution['Label'] = final_predictions

In [255]:
sample_solution.to_csv('audio_binary_classification_predict-5.csv', index=False)